In [16]:
import pyspark  
import regex
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 09:11:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import pandas as pd 
import numpy as np 

# Create a spark data frame that contains your favorite programming languages.

In [26]:
dicts = {'languages': [ "python",'c++','java','r','javascript']}

In [27]:
df = pd.DataFrame(dicts)

In [28]:
df = spark.createDataFrame(df)

In [29]:
# column name
df.languages

Column<'languages'>

In [30]:
# schema
df.printSchema()

root
 |-- languages: string (nullable = true)



In [31]:
# output shape
df

DataFrame[languages: string]

In [32]:
# show first 5 records
df.show(5)

+----------+
| languages|
+----------+
|    python|
|       c++|
|      java|
|         r|
|javascript|
+----------+



# Load the mpg dataset as a spark dataframe.

In [34]:
from pydataset import data
mpg = spark.createDataFrame(data('mpg'))


In [35]:
mpg.show(1)

+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|   trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|auto(l5)|  f| 18| 29|  p|compact|
+------------+-----+-----+----+---+--------+---+---+---+---+-------+
only showing top 1 row



In [48]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [54]:
# Create 1 column of output that contains a message like the one below:
# The 1999 audi a4 has a 4 cylinder engine.

In [64]:
mpg.select(concat(lit('The'),mpg.year, lit(' '), mpg.manufacturer, lit(' has a '), mpg.cyl, lit(' cylinder engine')).alias('description')).show(truncate = False)


+-----------------------------------------+
|description                              |
+-----------------------------------------+
|The1999 audi has a 4 cylinder engine     |
|The1999 audi has a 4 cylinder engine     |
|The2008 audi has a 4 cylinder engine     |
|The2008 audi has a 4 cylinder engine     |
|The1999 audi has a 6 cylinder engine     |
|The1999 audi has a 6 cylinder engine     |
|The2008 audi has a 6 cylinder engine     |
|The1999 audi has a 4 cylinder engine     |
|The1999 audi has a 4 cylinder engine     |
|The2008 audi has a 4 cylinder engine     |
|The2008 audi has a 4 cylinder engine     |
|The1999 audi has a 6 cylinder engine     |
|The1999 audi has a 6 cylinder engine     |
|The2008 audi has a 6 cylinder engine     |
|The2008 audi has a 6 cylinder engine     |
|The1999 audi has a 6 cylinder engine     |
|The2008 audi has a 6 cylinder engine     |
|The2008 audi has a 8 cylinder engine     |
|The2008 chevrolet has a 8 cylinder engine|
|The2008 chevrolet has a 8 cylin

In [66]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_extract, regexp_replace


In [ ]:
regexp_extract("address", r"^(\d+)", 1).alias("street_no"),


In [79]:
# Transform the trans column so that it only contains either manual or auto.
mpg.withColumn('trans', when(mpg.trans.like('auto%'), 'auto').otherwise('manual')).select('trans').show(5)

+------+
| trans|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
+------+
only showing top 5 rows



# Load the tips dataset as a spark dataframe.

In [81]:
tips = spark.createDataFrame(data('tips'))

In [82]:
# What percentage of observations are smokers?
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [90]:
tips.select(count(tips.smoker)).show()

+-------------+
|count(smoker)|
+-------------+
|          244|
+-------------+



In [97]:
tips.groupBy("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [100]:
from pyspark.sql.functions import col, expr

In [103]:
tips.groupby('smoker').count().withColumn(
    'percent',
        col('count') / tips.count() * 100).show()


+------+-----+------------------+
|smoker|count|           percent|
+------+-----+------------------+
|    No|  151|61.885245901639344|
|   Yes|   93|38.114754098360656|
+------+-----+------------------+



In [117]:
# Create a column that contains the tip percentage
tips.show(2)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 2 rows



In [122]:
tips.select(( tips.tip / tips.total_bill ).alias('tip percentage')).show(10)

+-------------------+
|     tip percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
+-------------------+
only showing top 10 rows



In [ ]:
# 

In [129]:
# Calculate the average tip percentage for each combination of sex and smoker.
tips.groupBy('sex', 'smoker').count().show()




+------+------+-----+
|   sex|smoker|count|
+------+------+-----+
|  Male|    No|   97|
|  Male|   Yes|   60|
|Female|    No|   54|
|Female|   Yes|   33|
+------+------+-----+



In [148]:
import pyspark.sql.functions as F

In [154]:
tips.withColumn('tip_percentage', tips.tip / tips.total_bill).groupby(
    'sex','smoker').agg(
        F.round(
            F.mean('tip_percentage'),4).alias(
                'avg_tip_p')).show()

+------+------+---------+
|   sex|smoker|avg_tip_p|
+------+------+---------+
|  Male|    No|   0.1607|
|  Male|   Yes|   0.1528|
|Female|    No|   0.1569|
|Female|   Yes|   0.1822|
+------+------+---------+



# Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [155]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [166]:
weather.withColumn('fahrenheit_', F.round((weather.temp_max * 1.8) + 32)).show()

+----------+-------------+--------+--------+----+-------+-----------+
|      date|precipitation|temp_max|temp_min|wind|weather|fahrenheit_|
+----------+-------------+--------+--------+----+-------+-----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|       55.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|       51.0|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|       53.0|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|       54.0|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|       48.0|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|       40.0|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|       45.0|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|       50.0|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|       49.0|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|       43.0|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|       43.0|
|2012-01-12|        

In [167]:
weather.withColumn('fahrenheit_min', F.round((weather.temp_min * 1.8) + 32)).show()

+----------+-------------+--------+--------+----+-------+--------------+
|      date|precipitation|temp_max|temp_min|wind|weather|fahrenheit_min|
+----------+-------------+--------+--------+----+-------+--------------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|          41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|          37.0|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|          45.0|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|          42.0|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|          37.0|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|          36.0|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|          37.0|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|          37.0|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|          41.0|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|          33.0|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|  

In [185]:
# Which month has the most rain, on average?
weather.withColumn('month', F.month(weather.date)).groupBy(F.col('month')).agg(F.mean(weather.precipitation).alias('avg_rain')).sort(F.col('avg_rain')).show()



+-----+-------------------+
|month|           avg_rain|
+-----+-------------------+
|    7|0.38870967741935486|
|    6| 1.1075000000000002|
|    8| 1.3201612903225806|
|    5| 1.6733870967741935|
|    9| 1.9624999999999997|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|    1| 3.7580645161290316|
|   10|  4.059677419354839|
|    3|  4.888709677419355|
|   12|  5.021774193548388|
|   11|  5.354166666666667|
+-----+-------------------+



In [187]:
# Which year was the windiest?
weather.withColumn('year', F.year(weather.date)
).groupby(F.col('year')).agg(F.mean(weather.wind).alias('avg_wind')).sort(
            F.col('avg_wind').desc()
            ).first()

Row(year=2012, avg_wind=3.400819672131147)

In [188]:
# What is the most frequent type of weather in January?
weather.filter(
    F.month(weather.date) == 1
    ).groupby(
    weather.weather
    ).count().sort(
        F.col('count').desc()
        ).show()


+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [196]:
# What is the average high and low temperature on sunny days in July in 2013 and 2014?
(weather.filter(F.month("date") == 7)
    .filter(F.year("date") > 2012)
    .filter(F.year("date") < 2015)
    .filter(F.col("weather") == F.lit("sun"))
    .agg(
        F.avg("temp_max").alias("average_high_temp"),
        F.avg("temp_min").alias("average_low_temp"),
    )
    .show())

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|26.828846153846158|14.18269230769231|
+------------------+-----------------+



In [194]:
# What percentage of days were rainy in q3 of 2015?
(weather.filter(F.year('date') == 2015)
    .filter(F.quarter('date') == 3)
    .select(
        F.when(
            F.col('weather') == 'rain', 1
            ).otherwise(0).alias('rain'))
    .agg(F.mean('rain')).show())


+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



In [193]:
# For each year, find what percentage of days it rained (had non-zero precipitation).
(weather.withColumn('year', F.year('date')).select(
        F.when(
            F.col('precipitation') > 0, 1
            ).otherwise(0).alias('did_rain'), 'year'
            )
    .groupby('year')
    .agg(F.mean('did_rain'))
    .show())

+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

